# Week 9 — Introduction to Databases and SQL

## Objectives- Create and query a SQLite database.- Use parameterized queries.- Practice basic SELECT/INSERT/UPDATE.

### Setup and Table Creation

In [ ]:
import sqlite3conn = sqlite3.connect(':memory:')cur = conn.cursor()cur.execute('CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, age INTEGER)')cur.executemany('INSERT INTO users (name, age) VALUES (?, ?)', [('Ana',30),('Bob',25),('Cara',35)])conn.commit()cur.execute('SELECT * FROM users')print(cur.fetchall())

**Activity:** Insert a new user using parameters and query all users older than 28.

In [ ]:
pass

### Updates and Filters

In [ ]:
cur.execute('UPDATE users SET age = ? WHERE name = ?', (26, 'Bob'))conn.commit()cur.execute('SELECT name, age FROM users WHERE age >= ?', (30,))print(cur.fetchall())

**Activity:** Delete users younger than 26 and show remaining names.

In [ ]:
pass

### RecapSQLite enables local SQL practice with parameterized queries for safety.